In [6]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [7]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('DiabetesBinaryClassification.csv')
# df = pd.read_csv('data.csv', header=None)

# Preprocessing data
df = df.drop_duplicates()
df = df.dropna()


X = []
y = []
for data in df.values:
    X.append( np.array( data[ : -1 ], dtype = float ) )
    y.append( [ data[ -1 ] ] )

    from sklearn.model_selection import train_test_split

x = np.asarray( X )
y = np.asarray( y )

for idn,n in np.ndenumerate(y):
  if n == 0:
    y[idn] = -1

# X = np.array(df.iloc[:, [0,1]])
# y = np.array(df.iloc[:, 2])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)

In [8]:
from cvxopt import matrix, solvers

def fit_model(X_train: np.array, y_train: np.array, C: float=0.1) -> tuple[int,int]:

  f = open( 'their.txt', 'w' )
  f.write( str( X_train ) + '\n' )
  f.write( str( y_train ) + '\n' )
  f.close()
  X_train = np.asarray( X_train )
  m, n = X_train.shape
  K = np.matmul(X_train, X_train.T)
  P = matrix(np.matmul(y_train,y_train.T) * K)
  q = matrix(np.ones((m, 1)) * -1)
  A = matrix((y_train.reshape(1, -1)))
  b = matrix(np.zeros(1))          
  G = matrix(np.vstack((np.eye(m) * -1, np.eye(m))))        
  h = matrix(np.hstack((np.zeros(m), np.ones(m) * C)))

  sol = solvers.qp(P, q, G, h, A, b)
  alphas = np.array(sol["x"])

  w = np.dot((y_train * alphas).T, X_train)[0]
  S = (alphas > 1e-5).flatten()
  b = np.mean(y_train[S] - np.dot(X_train[S], w.reshape(-1,1)))
  return w,b

w,b = fit_model(X_train, y_train)

     pcost       dcost       gap    pres   dres
 0: -1.4017e+02 -1.0253e+02  4e+03  3e+01  2e-10
 1: -2.2325e+01 -9.7954e+01  2e+02  1e+00  2e-10
 2: -1.7448e+01 -5.5815e+01  5e+01  2e-01  3e-11
 3: -1.8497e+01 -2.4736e+01  7e+00  2e-02  1e-11
 4: -2.0349e+01 -2.2019e+01  2e+00  4e-03  2e-11
 5: -2.0820e+01 -2.1397e+01  6e-01  9e-04  2e-11
 6: -2.0946e+01 -2.1230e+01  3e-01  4e-04  2e-11
 7: -2.1035e+01 -2.1111e+01  8e-02  7e-05  2e-11
 8: -2.1058e+01 -2.1083e+01  3e-02  2e-05  1e-11
 9: -2.1067e+01 -2.1072e+01  5e-03  3e-06  2e-11
10: -2.1069e+01 -2.1070e+01  6e-04  1e-07  2e-11
11: -2.1069e+01 -2.1069e+01  1e-05  3e-09  2e-11
Optimal solution found.


In [9]:
from sklearn import metrics

def evaluate_model(X_test: np.array, y_test: np.array, w: np.array, b: float):
  X_test = np.asarray( X_test )
  n_test = X_test.shape[0]
  prod = np.dot(X_test,w) + np.full(n_test,b)
  y_pred = np.sign(prod)

  # Model Accuracy: how often is the classifier correct?
  print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

evaluate_model(X_test, y_test, w, b)

Accuracy: 0.7734375


In [10]:
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))

/home/manojturaga/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.7838541666666666
Precision: 0.6699029126213593
Recall: 0.5847457627118644
